In [1]:
### Task 3 ###

from google.cloud import bigquery
from google.oauth2 import service_account
import sqlite3
import pandas as pd
import pandas_gbq

service_path = "/Users/ethanmeersman/ADA/wedge project/clean-wedge-files"
service_file =  '/wedge project/clean-wedge-files-b559ec63983a.json'
gbq_proj_id = 'clean-wedge-files'
gbq_dataset_id = 'Clean_Wedge'

credentials = service_account.Credentials.from_service_account_file("/Users/ethanmeersman/ADA/wedge project/clean-wedge-files-b559ec63983a.json")

client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [2]:
import sqlite3
db = sqlite3.connect("WEDGEE.db")
cur = db.cursor()

In [11]:
# Query for question 1 (sales by date by hour):

query_one = """
SELECT
  EXTRACT(DATE
  FROM
    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(timestamp_field_0))) AS date,
  EXTRACT(HOUR
  FROM
    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(timestamp_field_0))) AS hour,
  SUM(double_field_14) AS total_spend,
  count(distinct(date(timestamp_field_0) || int64_field_1 || int64_field_2 || int64_field_3)) as num_trans,
  sum(CASE WHEN (string_field_8 = 'V' or string_field_8 = 'R') THEN -1 ELSE 1 END) as num_items
FROM
  `clean-wedge-files.Clean_Wedge.wedge_*`
WHERE int64_field_9 != 0 and
      int64_field_9 != 15 and
      string_field_8 != 'M' and
      string_field_8 != 'C' and
      string_field_8 != 'J' and
      (string_field_8 = '' or 
      string_field_8 = ' ' or 
      string_field_8 = 'V' or 
      string_field_8 = 'R')
GROUP BY
  date,
  hour
ORDER BY
  date ASC,
  hour ASC
"""

In [12]:
df_one = (
    client.query(query_one)
    .result()
    .to_dataframe(
        create_bqstorage_client=True,
    )
)

# Open connection to (and create) database
db = sqlite3.connect("WEDGEE.db")

# Export pandas dataframe to database
df_one.to_sql("sales_date_hour", db, if_exists="replace")

# Close database
db.close()

In [13]:
# Query for question 2 (sales by owner by month):

query_two = """
SELECT
  int64_field_45,
  EXTRACT(YEAR
  FROM
    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(timestamp_field_0))) AS year,
  EXTRACT(MONTH
  FROM
    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(timestamp_field_0))) AS month,
  SUM(double_field_14) AS sales,
  count(distinct(date(double_field_14) || int64_field_1 || int64_field_2 || int64_field_3)) as transactions,
  sum(CASE WHEN (string_field_8 = 'V' or string_field_8 = 'R') THEN -1 ELSE 1 END) as items
FROM
  `clean-wedge-files.Clean_Wedge.wedge_*`
WHERE int64_field_9 != 0 and
      int64_field_9 != 15 and
      string_field_8 != 'M' and
      string_field_8 != 'C' and
      string_field_8 != 'J' and
      (string_field_8 = '' or 
      string_field_8 = ' ' or 
      string_field_8 = 'V' or 
      string_field_8 = 'R')
GROUP BY
  card_no,
  year,
  month
ORDER BY
 card_no,
  year ASC,
  month ASC
"""

In [14]:
# Send query to GBQ and export results as pandas data frame
df_two = (
    client.query(query_two)
    .result()
    .to_dataframe(
        create_bqstorage_client=True,
    )
)

# Open connection to (or create) database
db = sqlite3.connect("WEDGEE.db")

# Export pandas dataframe to database
df_two.to_sql("sales_owner_year_month", db, if_exists="replace")

# Close database
db.close()

BadRequest: 400 No matching signature for function DATE for argument types: FLOAT64. Supported signatures: DATE(TIMESTAMP, [STRING]); DATE(DATETIME); DATE(INT64, INT64, INT64); DATE(DATE); DATE(STRING) at [11:18]

(job ID: b04f8b86-0c67-4c69-89f8-616d09fcb037)

                                         -----Query Job SQL Follows-----                                         

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:SELECT
   3:  int64_field_45,
   4:  EXTRACT(YEAR
   5:  FROM
   6:    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(timestamp_field_0))) AS year,
   7:  EXTRACT(MONTH
   8:  FROM
   9:    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(timestamp_field_0))) AS month,
  10:  SUM(double_field_14) AS sales,
  11:  count(distinct(date(double_field_14) || int64_field_1 || int64_field_2 || int64_field_3)) as transactions,
  12:  sum(CASE WHEN (string_field_8 = 'V' or string_field_8 = 'R') THEN -1 ELSE 1 END

In [15]:
table_id = "dept_lookup"
location_name = f"{gbq_proj_id}.{gbq_dataset_id}.{table_id}"

client.create_table(location_name)

Table(TableReference(DatasetReference('clean-wedge-files', 'Clean_Wedge'), 'dept_lookup'))

In [16]:
filename = "dept_lookup.csv"

dataset_ref = client.dataset(gbq_dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.autodetect = True
job_config.schema = [
    bigquery.SchemaField("dept_num", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("dept_name", "STRING", mode="NULLABLE")]


with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

FileNotFoundError: [Errno 2] No such file or directory: 'dept_lookup.csv'

In [17]:
# Query for question 3 (sales by product by year by month):

query_three = """
SELECT
  string_field_4,
  string_field_5,
  int64_field_9,
  dept_name,
  EXTRACT(YEAR
  FROM
    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(timestamp_field_0))) AS year,
  EXTRACT(MONTH
  FROM
    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(timestamp_field_0))) AS month,
  SUM(total) AS sales,
  count(distinct(date(datetime) || register_no || emp_no || trans_no)) as transactions,
  sum(CASE WHEN (trans_status = 'V' or string_field_8 = 'R') THEN -1 ELSE 1 END) as items
FROM
  `clean-wedge-files.Clean_Wedge.wedge_*` as trans
LEFT JOIN `evident-catcher-327918.wedge_master.dept_lookup` as dept ON dept.dept_num = trans.department
WHERE string_field_9 != 0 and
      string_field_9 != 15 and
      string_field_8 != 'M' and
      string_field_8 != 'C' and
      string_field_8 != 'J' and
      (string_field_8 = '' or 
      string_field_8 = ' ' or 
      string_field_8 = 'V' or 
      string_field_8 = 'R')
GROUP BY
  string_field_4,
  string_field_5,
  int64_field_9,
  dept_name,
  year,
  month
ORDER BY
  string_field_5,
  year ASC,
  month ASC
"""

In [18]:
df_three = (
    client.query(query_three)
    .result()
    .to_dataframe(
        create_bqstorage_client=True,
    )
)

# Open connection to (or create) database
db = sqlite3.connect("WEDGEE.db")

# Export pandas dataframe to database
df_three.to_sql("sales_product_year_month", db, if_exists="replace")

# Close database
db.close()

Forbidden: 403 Access Denied: Table evident-catcher-327918:wedge_master.dept_lookup: User does not have permission to query table evident-catcher-327918:wedge_master.dept_lookup.

(job ID: f4bd056d-bb4c-460b-b60e-1f0bd1c29326)

                                      -----Query Job SQL Follows-----                                       

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:SELECT
   3:  string_field_4,
   4:  string_field_5,
   5:  int64_field_9,
   6:  dept_name,
   7:  EXTRACT(YEAR
   8:  FROM
   9:    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(timestamp_field_0))) AS year,
  10:  EXTRACT(MONTH
  11:  FROM
  12:    PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S %Z', STRING(timestamp_field_0))) AS month,
  13:  SUM(total) AS sales,
  14:  count(distinct(date(datetime) || register_no || emp_no || trans_no)) as transactions,
  15:  sum(CASE WHEN (trans_status = 'V' or string_field_8 = 'R') THEN -1 ELSE 1 END) as item